In [ ]:
from agents import Agent, trace, WebSearchTool, Runner, function_tool
from agents.model_settings import ModelSettings
import os
from dotenv import load_dotenv


In [6]:
load_dotenv()

True

In [7]:
# THe System prompt for the agent
Instruction="You are a research agent. Based on the message provided by the user, you must research the web using websearch tool.\
     capture only the main points and write in 2 to 3 sentenecs. Mesaningful sentences are enough and no need to worry about the grammar. \
     Each paragraph should be less than 300 works. This will be consumed by someone synthesizing a report, so its vital you capture the \
     essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

In [8]:
from agents import ModelSettings


searchagent = Agent(
    name="searchagent",
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",   # DeepSeek model name
    instructions=Instruction,
    model_settings=ModelSettings(tool_choice="required")
)

In [9]:
with trace("Running Agent"):
    result = await Runner.run(searchagent, "Best AI Startup ideas with zero or low cost")


In [ ]:
from pydantic import BaseModel

In [16]:
# Now lets create a search assistancce tool that gives us 3 different searches based on the input provided by the user
from pydantic import BaseModel
from typing import List

how_manu_searches=3
Instructions=f" You need to come up with {how_manu_searches} searches based on the query given to you"

# lets create pydantec basemodel class for output

class websearchitem(BaseModel):
     reason: str
     query: str

class websearchlist(BaseModel):
    searches: List[websearchitem]

helpingagent=Agent(
    name="helpingagent",
     model="gpt-4o-mini",
     instructions=Instructions,
     output_type=websearchlist
)

In [32]:
# Now lets feed these searcches to our search agent

# We must write functions and call them 

async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(helpingagent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    print (result.final_output)
    return result.final_output


In [34]:
import asyncio

In [37]:
# Now time to feed each single search item in actual search agent using a function 
# first function using asyncio, taking the list from the previous func, and then adding it together as tasks which has a search function but it is not executed unless it is called
# in the next search function.  the task is run using gather.

async def perform_searches(search_plan: websearchlist):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: websearchitem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(searchagent, input)
    return result.final_output


In [59]:
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
import sendgrid

In [ ]:
# Now lets create some function_tools and we can use guardrail and  in the same lab test
from sendgrid.helpers.mail import email
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("iqra@iqratech.site") # Change this to your verified email
    to_email = To("iqraagenticai@gmail.com") # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}




In [67]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""



emailsender=Agent(
    name="emailsender",
    model="gpt-4o-mini",
    tools=[send_email],
    instructions=INSTRUCTIONS

)

In [52]:
# Now lets create Manager or analyser agent
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str 

    markdown_report: str 

    follow_up_questions: list[str] 


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

In [48]:
from agents import input_guardrail, GuardrailFunctionOutput

In [49]:
class guard_check(BaseModel):
    is_name_found: bool
    found_name: str

# Create an agent to give the output
guard_agent=Agent(
    name="Guard Agent",
    model="gpt-4o-mini",
    instructions="Check if the input of the user contains personal names in what you want to do",
    output_type=guard_check
)

@input_guardrail
async def guardrail_func(ctx, agent, message):

     # Run the agent in this such as superevisor checking the xray machine and turning it on.
    result=await Runner.run(guard_agent, message, context=ctx.context)

     # Check if name exist and return the name
    is_name_found=result.final_output.is_name_found # Check if name exist and return the name
    found_name=result.final_output
    return GuardrailFunctionOutput(output_info=found_name, tripwire_triggered=is_name_found)

In [71]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(emailsender, report.markdown_report)
    print("Email sent")
    return report

In [ ]:
nextresult=""
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    nextresult=report
    await send_email(report)  
    print("Hooray!")

In [72]:
# Just send the email with the existing report, no new searches
with trace("Email retry"):
    print("Sending email with existing report...")
    await send_email(nextresult)  
    print("Email sent successfully!")

Sending email with existing report...
Writing email...
Email sent
Email sent successfully!
